In [59]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from IPython.display import display

%matplotlib inline

In [60]:
# Read data from .csv files 

loc = "voetbal_1956-20220309/"

raw_cards_file = pd.read_csv (loc + '_cards.csv')
raw_goals_file = pd.read_csv (loc + '_goals.csv')
raw_matches_file = pd.read_csv (loc + '_matches.csv')
raw_players_file = pd.read_csv (loc + '_players.csv')
raw_subs_file = pd.read_csv (loc + '_subs.csv')

In [61]:
df_card_filtered = raw_cards_file[(raw_cards_file['type'] == 'rode kaart') | 
                                  (raw_cards_file['type'] == 'gele kaart') |
                                  (raw_cards_file['type'] == 'gele') |
                                  (raw_cards_file['type'] == 'kaartgele') |
                                  (raw_cards_file['type'] == 'rode') |
                                  (raw_cards_file['type'] == 'kaartrode')  ]
            
df_card_filtered["type"].replace({"rode kaart": 2, "gele kaart": 1, "gele": 1 , "kaartgele" : 1 , "rode":2, "kaartrode" : 2 }, inplace=True)

filterHomeTeam = df_card_filtered[ (df_card_filtered['home'] == df_card_filtered['team'])]
filterAwayTeam = df_card_filtered[ (df_card_filtered['away'] == df_card_filtered['team'])]

In [62]:
# Away yellow cards
filterAwayYellowCard = filterAwayTeam[(filterAwayTeam['type'] == 1)]
groupByAwayYellowCard = filterAwayYellowCard.groupby('away').sum()
groupByAwayYellowCard['type'].head(10)

away
ado-den-haag           704
afc-ajax               780
ajax-amsterdam          73
az-alkmaar             825
cambuur-leeuwarden      75
de-graafschap          473
excelsior-rotterdam     81
fc-amsterdam            17
fc-den-bosch           212
fc-dordrecht           124
Name: type, dtype: int64

In [63]:
# Away red cards
filterAwayRedCard = filterAwayTeam[(filterAwayTeam['type'] == 2)]
groupByAwayRedCard = filterAwayRedCard.groupby('away').sum()
groupByAwayRedCard['type'].head(10)

away
ado-den-haag           46
afc-ajax               54
ajax-amsterdam          8
az-alkmaar             68
cambuur-leeuwarden     12
de-graafschap          54
excelsior-rotterdam     6
fc-den-bosch           14
fc-dordrecht           14
fc-emmen                8
Name: type, dtype: int64

In [64]:
# Home yellow cards
filterHomeYellowCard = filterHomeTeam[(filterHomeTeam['type'] == 1)]
groupByHomeYellowCard = filterHomeYellowCard.groupby('home').sum()
groupByHomeYellowCard

,Unnamed: 0,round,minute,type
home,,,,
ado-den-haag,18471,8977,24635.0,534
afc-ajax,18296,6771,15369.0,394
ajax-amsterdam,3372,691,2419.0,47
az-alkmaar,50521,9067,25774.0,538
cambuur-leeuwarden,38841,886,1684.0,61
de-graafschap,45592,5886,15127.0,350
excelsior-rotterdam,25721,622,1980.0,34
fc-amsterdam,92,123,132.0,7
fc-den-bosch,16480,2400,3674.0,141


In [65]:
# Home red cards 
filterHomeRedCard = filterHomeTeam[(filterHomeTeam['type'] == 2)]
groupByHomeRedCard = filterHomeRedCard.groupby('home').sum()
groupByHomeRedCard

,Unnamed: 0,round,minute,type
home,,,,
ado-den-haag,501,294,830.0,30
afc-ajax,818,289,973.0,32
az-alkmaar,2254,448,1251.0,48
cambuur-leeuwarden,493,26,80.0,2
de-graafschap,2105,295,855.0,30
excelsior-rotterdam,1563,49,91.0,4
fc-den-bosch,576,87,243.0,8
fc-dordrecht,380,55,315.0,10
fc-emmen,261,14,112.0,4
